Google Map Scrapping Without API (Using Selenium Web Driver, beautifulsoup and Python Scripting)






<h2>$\color{green}{\text{Importing Libraries}}$</h2>


In [ ]:

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from selenium.webdriver import ActionChains
from selenium.webdriver.common.touch_actions import TouchActions
from selenium.webdriver.common.keys import Keys 


import io
import csv
import json
import requests
from bs4 import BeautifulSoup
import time

import pandas as pd
import numpy as np



<h3>$\color{black}{\text{Waiting time between operations & misc.}}$</h3>


In [ ]:
#Waiting time between operations 

startup_delay=5
next_page_delay = 3.5
wait_after_element_loaded= 2
delay_to_check_if_element_visible = 1
load_specific_result_delay = 4

csv_file = "Restaurants_in_Bangladesh.csv"
csv_columns = ['name','rating','total_rating_given','price_level','address']

map_query= "https://www.google.com/maps/search/restaurants+in+Dhaka/&pws=0"
map_query2= "https://www.google.com/maps/search/restaurants+in+Chittagong/&pws=0"
map_query3= "https://www.google.com/maps/search/restaurants+in+Rajshahi/&pws=0"
map_query4= "https://www.google.com/maps/search/restaurants+in+Barishal/&pws=0"
map_query5= "https://www.google.com/maps/search/restaurants+in+Khulna/&pws=0"
map_query6= "https://www.google.com/maps/search/restaurants+in+Rangpur/&pws=0"
map_query7= "https://www.google.com/maps/search/restaurants+in+Mymensingh/&pws=0"
map_query8= "https://www.google.com/maps/search/restaurants+in+Sylhet/&pws=0"



<h3>$\color{blue}{\text{Defining extractor functions.}}$</h3>


In [ ]:
table=[]
def google_maps_extractor(driver,url):
#     driver.get('chrome://settings/')
#     driver.execute_script('chrome.settingsPrivate.setDefaultZoom(0.50);')
    driver.get(url)

    
    #Intializing with the assumption that there are more than one page result
  

    more_pages = True
    master_restaurant_array = []
    ctr = 0
    
    while more_pages:
        time.sleep(next_page_delay) #takes_few_seconds_to_go_next_page
        

        html = driver.page_source
        
        divs = WebDriverWait(driver,3).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR,"div"))) #wait_till_loading
        
        soup = BeautifulSoup(html,'lxml')
        
        time.sleep(wait_after_element_loaded)

        result_list = soup.find_all('div',attrs={'class': 'V0h1Ob-haAclf OPZbO-KE6vqe o0s21d-HiaYvf'})
        #         result_list_span = soup.find_all('span',attrs={'class': 'ZkP5Je'})
#         result_scroll_panel = driver.find_element_by_css_selector("div[class^='siAUzd-neVct section-scrollbox cYB2Ge-oHo7ed cYB2Ge-ti6hGc siAUzd-neVct-Q3DXx-BvBYQ']")
        
        
        for result_val in result_list:
            
            cleaned_data={}   #Empty_dictionary
          


            div = result_val.find("div", class_="qBF1Pd gm2-subtitle-alt-1")
            name= div.span.text
          
    
            rating= result_val.find('span',class_='MW4etd')
        
            if rating !=None:
                rating = rating.text
            else:
                rating='0'
            total_rating_given= result_val.find('span',class_='UY7F9')
            
            #checking if any info is not provided : 'None'
            
            if total_rating_given !=None:
                total_rating_given=total_rating_given.text
            else:
                total_rating_given ='0'
            
            price_level=result_val.find('span', attrs={'jsan':"0.aria-label,0.role"})

            if price_level !=None:
                price_level=price_level.text
            else:
                price_level = "not specified"
            referenceSpan= result_val.find('span',class_='bXlT7b-hgDUwe')
            if referenceSpan!=None:
                address = referenceSpan.find_next_sibling('span')

            else:
                address = None
            
            
            if address!=None:
                address=address.text.strip()
            else:
                address="Not Provided"



            cleaned_data["name"] = name
            cleaned_data["rating"] = rating.replace("(", "").replace(")","").replace("-","").strip()        
            cleaned_data["total_rating_given"] = total_rating_given.replace("(", "").replace(")","").replace("-","").strip()          
            cleaned_data["price_level"] = price_level
            cleaned_data["address"] = address


            print(f"******Name:{name}********")
            print(f"rating:{rating}")
            print(f"total_rating_given:{total_rating_given}")
            print(f"price_level:{price_level}")
            print(f"address:{address}")
            print("")
                                 
            
#             try:
#                 with open(csv_file, 'a', encoding='utf-8') as csvfile:
#                     writer = csv.DictWriter(csvfile, fieldnames=csv_columns, delimiter='|')
                    
#                     writer.writerow(cleaned_data)
#             except IOError:
#                 print("I/O Error")
                
            
#              ActionChains(driver).click(back_btn).perform()
#              time.sleep(back_to_main_list_delay)
            
            table.append(cleaned_data)
     
        next_btn = driver.find_element_by_xpath('//*[contains(@id, "ppdPk-Ej1Yeb-LgbsSe-tJiF1e")]')
        
#         DISCONNECTED_MSG = 'Unable to evaluate script'
#         if driver.get_log('driver')[-1]['message'] == DISCONNECTED_MSG:
#             print ("Browser window closed by user")
#         time.sleep(1)
        
        print(next_btn)
        
        prop = next_btn.get_property('disabled')


        if prop == True:
            more_pages=False

            break
        time.sleep(next_page_delay)
        ActionChains(driver).click(next_btn).perform()
        
        

                

In [ ]:
import warnings
warnings.filterwarnings('ignore')


In [ ]:
options = webdriver.ChromeOptions()
options.add_argument('--lang=en-GB') #Setting language to En-UK

driver = webdriver.Chrome(chrome_options=options)
time.sleep(startup_delay)
google_maps_extractor(driver,map_query,)

******Name:Shakib's 75 Restaurant********
rating:4.3
total_rating_given:(342)
price_level:not specified
address:Ruapyan ZR Plaza, Level 9, Satmasjid Road, Road 9/A, House 46 Dhaka

******Name:Family Kitchen********
rating:5.0
total_rating_given:(11)
price_level:not specified
address:29/1 Monowara Cotage, E Nayatola Rd

******Name:Izumi Japanese Kitchen********
rating:4.5
total_rating_given:(1,430)
price_level:$$$$
address:House no: 88, 24C Rd No 113/A

******Name:A La Greek - Mediterranean Cuisine********
rating:4.3
total_rating_given:(186)
price_level:not specified
address:Rangs Fortune Square, Level 6A Plot 32, Road, 02 Road No. 2

******Name:HEXA Dine - Restaurant & Café********
rating:4.4
total_rating_given:(340)
price_level:not specified
address:House: 34 (1st Floor), Green Regency, Road No: 10, Block- D

******Name:BAO********
rating:4.2
total_rating_given:(902)
price_level:$$
address:House 1/A 1st floor, Unit-2, রোড নং # ২৩

******Name:Bistro-E********
rating:4.5
total_rating_gi

******Name:Olea********
rating:4.2
total_rating_given:(67)
price_level:$$
address:Le Méridien Dhaka 79/A Commercial Area Dhaka

******Name:SEAGULL RESTAURANT.********
rating:4.1
total_rating_given:(638)
price_level:$$
address:116 Culvert Road

******Name:Dhaka foods & Restaurant********
rating:3.8
total_rating_given:(228)
price_level:$$
address:Shapla Building, Near, south

******Name:Sunami Restaurant********
rating:3.8
total_rating_given:(1,504)
price_level:$$
address:25/1

******Name:Chows********
rating:4.2
total_rating_given:(821)
price_level:$$$$
address:6 Rd No. 19/A

******Name:Lake Terrace********
rating:4.0
total_rating_given:(2,869)
price_level:$$$
address:House # 25/E, Lake Dr Rd

******Name:Fakhruddin Restaurant********
rating:3.9
total_rating_given:(2,416)
price_level:$$
address:37, South Gulshan, Circle-1

******Name:Elements Restaurant, InterContinental Dhaka********
rating:4.6
total_rating_given:(463)
price_level:not specified
address:1 Minto Rd

******Name:Basilico***

******Name:Roll Xpress Cafe********
rating:4.4
total_rating_given:(1,376)
price_level:$$
address:House 34, Block B Rd No # 21

******Name:Cafe Euphoria********
rating:3.8
total_rating_given:(245)
price_level:$$
address:100 Rd No. 11

******Name:Thai Signatures********
rating:4.3
total_rating_given:(448)
price_level:$$
address:House 6/A Rd 113

******Name:Nirob Hotel********
rating:3.9
total_rating_given:(2,378)
price_level:$$
address:H.No 114 নাজিমুদ্দিন রোড, Near Old Central Jail

******Name:Dark Burg Dhaka********
rating:4.9
total_rating_given:(15)
price_level:$$
address:Sultan Mansion, Signature Food Court Roof Top (8th Floor) Holding No. 3, 12 Begum Rokeya Avenue

******Name:Sky View Restaurant********
rating:3.9
total_rating_given:(465)
price_level:$$
address:Novo Tower, 270, 15th Floor, Shaheed Tajuddin Ahmed Ave

******Name:New Melbourne Thai & Chinese Restaurant********
rating:3.7
total_rating_given:(389)
price_level:not specified
address:220/A/2/1, বেগম রোকেয়া সরণী, West Kafr

<selenium.webdriver.remote.webelement.WebElement (session="6f4c02025cf5685b64f2ce202308a9ee", element="e999bc2e-94e6-4fd8-bd8e-9d09103426b7")>
******Name:Grand Prince Thai & Chinese Restaurant, Dhaka.********
rating:3.9
total_rating_given:(1,216)
price_level:$$
address:PARADISE PLAZA, PLOT NO. 6 & 11 MIRPUR-1,

******Name:Dhakaiya Pakki Biriyani********
rating:2.5
total_rating_given:(2)
price_level:$$
address:Road-12, House 37,Dhaka

******Name:Alitto Food********
rating:4.7
total_rating_given:(10)
price_level:$$$
address:Plot # 1, 2 ব্লক#ডি

******Name:The Dining Lounge Mirpur********
rating:3.7
total_rating_given:(719)
price_level:$$
address:Opens at 12:00 PM

******Name:Lake Green Lounge & Restaurant********
rating:3.7
total_rating_given:(250)
price_level:$$
address:Plot No-39, Block-C Aftab Nagar Main Rd

******Name:The Garden Kitchen at Sheraton Dhaka********
rating:5.0
total_rating_given:(1)
price_level:$$
address:44 Kemal Ataturk Ave

******Name:The Cafe Rio, Uttara********
rati

In [ ]:
options = webdriver.ChromeOptions()
options.add_argument('--lang=en-GB') #Setting language to En-UK

driver = webdriver.Chrome(chrome_options=options)
time.sleep(startup_delay)
google_maps_extractor(driver,map_query5,)

******Name:Vibez Cafe N Restaurant********
rating:4.0
total_rating_given:(384)
price_level:$$
address:83 S Central Rd

******Name:Saffron********
rating:4.0
total_rating_given:(79)
price_level:not specified
address:Opposite of Pioneer girl's College, 1 S Central Rd

******Name:ROVERS CAFE********
rating:4.3
total_rating_given:(951)
price_level:$$
address:Upper Jessore Rd

******Name:Mughal's Dine********
rating:3.9
total_rating_given:(702)
price_level:$$
address:69,Cemetery Road, Dak Bangla

******Name:ফায়ারফ্লাই রেস্টুরেন্ট********
rating:4.1
total_rating_given:(606)
price_level:$$
address:QGMR+V27

******Name:New Azmiri Kacchi Kitchen********
rating:4.0
total_rating_given:(460)
price_level:not specified
address:1sher e, bangla road

******Name:Deshi Kitchen********
rating:4.1
total_rating_given:(1,185)
price_level:$$
address:12, কেডিএ এভিনিউ, তমালতলা - শৈলকুপা সড়ক

******Name:Royal Kitchen********
rating:4.1
total_rating_given:(18)
price_level:not specified
address:43, Puraton Jesso

******Name:Shanon Cafe and BBQ********
rating:3.8
total_rating_given:(19)
price_level:$$
address:RH2J+PC9

******Name:Al-Amin Chotpoti********
rating:4.7
total_rating_given:(11)
price_level:$$$
address:East Baniakhamer, 9th Gali B.K Main road, Khulna

******Name:Aziztawsif Best Restaurent in Khulna********
rating:0
total_rating_given:0
price_level:$$
address:RH87+3XX

******Name:River Side Cafe & Restaurant********
rating:3.5
total_rating_given:(13)
price_level:$$
address:Rupsha River

******Name:Chinese Garden********
rating:4.0
total_rating_given:(36)
price_level:$$
address:Khulna City

******Name:Ruhin Restaurant********
rating:5.0
total_rating_given:(2)
price_level:$$
address:QHVH+7R4

******Name:Ammes Kitchen********
rating:0
total_rating_given:0
price_level:not specified
address:118 Upper Jessore Rd

******Name:অলকা রেস্তোঁরা********
rating:3.8
total_rating_given:(207)
price_level:not specified
address:Dakbangla Mor, Upper Jessore Rd

******Name:লতিফ চটপটি********
rating:5.0
tota

******Name:Rasmi Mini Chinese Restaurant********
rating:5.0
total_rating_given:(4)
price_level:$$
address:QHPG+9WG

******Name:Adi Mollar Hotel********
rating:3.6
total_rating_given:(16)
price_level:$$$
address:RGHX+WRC

******Name:Shamim Hotel, Daulatpur********
rating:3.5
total_rating_given:(283)
price_level:$
address:Khulna - Jessore - Dhaka Hwy

******Name:দিলু ভাইয়ের পুড়ির হোটেল********
rating:5.0
total_rating_given:(1)
price_level:$$
address:7 Shipyard Rd

******Name:Pizza Bazar********
rating:3.5
total_rating_given:(108)
price_level:$$
address:Khan Jahan Ali Rd, P. T. I More

******Name:Bismillah store & fastfood********
rating:3.3
total_rating_given:(3)
price_level:$$
address:Road no : 132, Plot no: 04, prosova mor, Khulna. khalispur

<selenium.webdriver.remote.webelement.WebElement (session="7d3602ef0bf60c8c548b43af1f14b901", element="8145cb90-60e9-4b9b-a1fb-0c1e0392e006")>
******Name:Biriyani House********
rating:5.0
total_rating_given:(4)
price_level:$$
address:RH78+JMM

***

In [ ]:
df = pd.DataFrame(data=table)
df.to_csv("wfp_programs.csv", encoding='utf-8-sig')

In [ ]:
import googlemaps


In [ ]:
gmaps_key = googlemaps.Client(key = "my_key")


$\color{blue}{\text{Geocoding the scapped locations.}}$


In [ ]:
df["lat"] = None
df["lon"] = None


for i in range(0, len(df), 1):
    geocode_result = gmaps_key.geocode(df.iat[i,0])
    try:
        lat = geocode_result[0]["geometry"]["location"]["lat"]
        lon = geocode_result[0]["geometry"]["location"]["lon"]
        df.iat[i, df.cloumns.get_loc("lat")]=lat
        df.iat[i, df.cloumns.get_loc("lon")]=long
    except:
        lat= None
        lon=None